In [1]:
from easydict import EasyDict
# opt的使用请参照auxiliary/argument_parser.py中opt的说明
opt = {"normalization": "UnitBall",
            "class_choice": ["plane"], 
            "SVR": True, 
            "sample": True, 
            "npoints": 2500, 
            "number_points": 2500, 
            "shapenet13": True, 
            "demo": False, 
            "data_augmentation_axis_rotation": True,
            "data_augmentation_random_flips": False, 
            "batch_size": 32, 
            "workers": 0, 
            "batch_size_test": 32, 
            "random_translation": False, 
            "anisotropic_scaling": False, 
            "random_rotation": False, # 以上为加载dataset需要
            "multi_gpu": [0],
            "bottleneck_size": 1024,
            "nb_primitives": 1,
            "number_points": 2500,
            "number_points_eval": 2500,
            "remove_all_batchNorms": False,
            "template_type": "SPHERE",
            "dim_template": 3,
            "hidden_neurons": 512,
            "num_layers": 2,
            "activation" : "relu",
            "reload_model_path": "",
            "reload_decoder_path": "",
            "train_only_encoder": False,
            "lrate": 0.001,
            "no_learning": False,
            "nepoch": 150,
            "reload_optimizer_path": "", #下面两个是log信息
            "log_path": "log",
            "name": "test_code",
            "loop_per_epoch": 1,
            "lr_decay_1": 120,
            "lr_decay_2": 140,
            "lr_decay_3": 145,
            "visdom_port": 8890,
            "http_port": 8891,
            "env": "Atlasnet",
            "dir_name": "",
            "demo": False}
        
        

In [2]:
from dataset.trainer_dataset import TrainerDataset
Dataset = TrainerDataset()
Dataset.opt = EasyDict(opt)
Dataset.build_dataset()

Dataset normalization : UnitBall
Create Shapenet Dataset...
['02691156', '02747177', '02773838', '02801938', '02808440', '02818832', '02828884', '02834778', '02843684', '02858304', '02871439', '02876657', '02880940', '02924116', '02933112', '02942699', '02946921', '02954340', '02958343', '02992529', '03001627', '03046257', '03085013', '03207941', '03211117', '03261776', '03325088', '03337140', '03467517', '03513137', '03593526', '03624134', '03636649', '03642806', '03691459', '03710193', '03759954', '03761084', '03790512', '03797390', '03928116', '03938244', '03948459', '03991062', '04004475', '04074963', '04090263', '04099429', '04225987', '04256520', '04330267', '04379243', '04401088', '04460130', '04468005', '04530566', '04554684']
    category 02691156  airplane Number Files :3236


/root/anaconda3/envs/env1/lib/python3.7/site-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


    category 02828884  bench Number Files :1452
    category 02933112  cabinet Number Files :1257
    category 02958343  car Number Files :5996
    category 03001627  chair Number Files :5422
    category 03211117  display Number Files :876
    category 03636649  lamp Number Files :1854
    category 03691459  loudspeaker Number Files :1294
    category 04090263  rifle Number Files :1897
    category 04256520  sofa Number Files :2538
    category 04379243  table Number Files :6807
    category 04401088  telephone Number Files :841
    category 04530566  vessel Number Files :1551
Reload dataset : /mnt/d/Mr.Wang/大学/大三上/计算机视觉/Atlas/AtlasNet-master/data/cache/UnitBallTrueairplane_bench_cabinet_car_chair_display_lamp_loudspeaker_rifle_sofa_table_telephone_vessel
Dataset Size: 11674
Dataset normalization : UnitBall
Create Shapenet Dataset...
['02691156', '02747177', '02773838', '02801938', '02808440', '02818832', '02828884', '02834778', '02843684', '02858304', '02871439', '02876657', '0288094

In [5]:
# 注意metro部分需要修改地址
import os
import time
import torch
import pymesh
from training.trainer_loss import TrainerLoss
from model.trainer_model import TrainerModel
from dataset.trainer_dataset import TrainerDataset
from training.trainer_iteration import TrainerIteration
from dataset.trainer_dataset import TrainerDataset
import dataset.mesh_processor as mesh_processor
from termcolor import colored
import torch.optim as optim
import numpy as np

class Trainer(TrainerLoss, TrainerIteration, TrainerModel):
    def __init__(self, opt):
        super(Trainer, self).__init__()
        self.opt = opt
        self.opt.training_media_path = os.path.join(self.opt.dir_name, "training_media")
        self.dataset_train = None
        self.build_network()
        self.build_optimizer()
        self.build_losses()
        self.init_log()
        self.colormap = mesh_processor.ColorMap()

        self.flags = EasyDict()
        self.flags.train = False
        self.flags.media_count = 0

        self.iteration = 0
        self.epoch = 0

    def init_log(self):
        self.log_path = os.path.join(self.opt.log_path, self.opt.name, 'summary')
        if not os.path.exists(self.log_path):
            os.makedirs(self.log_path)
        self.log_name = os.path.join(self.opt.log_path, self.opt.name, 'loss_log.txt')
    
    def log_start_train(self):
        with open(self.log_name, "a") as log_file:
            now = time.strftime("%c")
            log_file.write('================ Training Loss (%s) ================\n' % now)

    def increment_iteration(self):
        self.iteration = self.iteration + 1
    def reset_iteration(self):
        self.iteration = 0
    def increment_epoch(self):
        self.epoch = self.epoch + 1
    def reset_epoch(self):
        self.epoch = self.opt.start_epoch

    def print_iteration_stats(self, loss):
        """
        print stats at each iteration
        """
        current_time = time.time()
        ellpased_time = current_time - self.start_train_time
        total_time_estimated = self.opt.nepoch * (self.opt.len_dataset / self.opt.batch_size) * ellpased_time / (
                0.00001 + self.iteration + 1.0 * self.epoch * self.opt.len_dataset / self.opt.batch_size)  # regle de 3
        ETL = total_time_estimated - ellpased_time
        print(
            f"\r["
            + colored(f"{self.epoch}", "cyan")
            + f": "
            + colored(f"{self.iteration}", "red")
            + "/"
            + colored(f"{int(self.opt.len_dataset / self.opt.batch_size)}", "red")
            + "] chamfer train loss:  "
            + colored(f"{loss.item()} ", "yellow")
            + colored(f"Ellapsed Time: {ellpased_time / 60 / 60}h ", "cyan")
            + colored(f"ETL: {ETL / 60 / 60}h", "red"),
            end="",
        )

    def learning_rate_scheduler(self):
        """
        Defines the learning rate schedule
        """
        # Warm-up following https://arxiv.org/pdf/1706.02677.pdf
        if len(self.next_learning_rates) > 0:
            next_learning_rate = self.next_learning_rates.pop()
            print(f"warm-up learning rate {next_learning_rate}")
            for g in self.optimizer.param_groups:
                g['lr'] = next_learning_rate

        # Learning rate decay
        if self.epoch == self.opt.lr_decay_1:
            self.opt.lrate = self.opt.lrate / 10.0
            print(f"First learning rate decay {self.opt.lrate}")
            self.optimizer = optim.Adam(self.network.parameters(), lr=self.opt.lrate)
        if self.epoch == self.opt.lr_decay_2:
            self.opt.lrate = self.opt.lrate / 10.0
            print(f"Second learning rate decay {self.opt.lrate}")
            self.optimizer = optim.Adam(self.network.parameters(), lr=self.opt.lrate)
        if self.epoch == self.opt.lr_decay_3:
            self.opt.lrate = self.opt.lrate / 10.0
            print(f"Third learning rate decay {self.opt.lrate}")
            self.optimizer = optim.Adam(self.network.parameters(), lr=self.opt.lrate)
            
    def demo(self, data1, demo_path, data2=None):
        """
        This function takes an image or pointcloud path as input and save the mesh infered by Atlasnet
        Extension supported are ply npy obg and png
        :return: path to the generated mesh
        """
        # ext = demo_path.split('.')[-1]
        # self.data = self.datasets.dataset_train.load(demo_path)
        self.data = data1
        self.data = EasyDict(self.data)

        if data2 is None:
            data2 = data1

        #prepare normalization
        get_normalization = EasyDict(data2)

        
        self.make_network_input()
        mesh = self.network.module.generate_mesh(self.data.network_input)
        if get_normalization.operation is not None:
            # Undo any normalization that was used to preprocess the input.
            vertices = torch.from_numpy(mesh.vertices).clone().unsqueeze(0)
            get_normalization.operation.invert()
            unnormalized_vertices = get_normalization.operation.apply(vertices)
            mesh = pymesh.form_mesh(vertices=unnormalized_vertices.squeeze().numpy(), faces=mesh.faces)

        if self.opt.demo:
            path = demo_path.split('.')
            path[-2] += "AtlasnetReconstruction"
            path[-1] = "ply"
            path = ".".join(path)
        else:
            path = '/'.join([self.log_path, str(self.flags.media_count)]) + ".ply"
            self.flags.media_count += 1

        print(f"Atlasnet generated mesh at {path}!")
        mesh_processor.save(mesh, path, self.colormap)
        return path
    


In [6]:
opt = EasyDict(opt)
opt.len_dataset = Dataset.datasets.len_dataset
opt.env = opt.env + opt.dir_name.split('/')[-1]
mod = Trainer(opt)
mod.start_train_time = time.time()

print(mod.log_path)

mod.flags.train = True
if not mod.opt.no_learning:
    mod.network.train()
else:
    mod.network.eval()
mod.learning_rate_scheduler()
mod.reset_iteration()
for i in range(mod.opt.loop_per_epoch):
    iterator = Dataset.datasets.dataloader_train.__iter__()
    mod.reset_iteration()
    mod.log_start_train()
    for item in iterator:
        mod.increment_iteration()
        mod.data = EasyDict(item)
        mod.data.points = mod.data.points.to(mod.opt.device)
        if Dataset.datasets.data_augmenter is not None and not Dataset.opt.SVR:
            Dataset.datasets.data_augmenter(Dataset.data.points)
        mod.train_iteration()
        # 利用demo函数检查结果 生成的ply文件保存在mod.log_path处
        demo_path = mod.data.image_path[0]
        demo_path = demo_path+'/01.png'
        demo_data = Dataset.datasets.dataset_train.load(demo_path)
        with torch.no_grad():
            mod.demo(EasyDict(demo_data), demo_path)
        break
        

New MLP decoder : hidden size 512, num_layers 2, activation relu
No network weights to reload!
log/test_code/summary
[0: 1/364] chamfer train loss:  0.12231758236885071 Ellapsed Time: 0.00023219810591803655h ETL: 12.705956467290983hAtlasnet generated mesh at log/test_code/summary/0.ply!


In [13]:
opt = EasyDict(opt)
opt.len_dataset = Dataset.datasets.len_dataset
opt.env = opt.env + opt.dir_name.split('/')[-1]
mod = Trainer(opt)
mod.start_train_time = time.time()

print(mod.log_path)

mod.flags.train = False
mod.network.eval()
mod.learning_rate_scheduler()
mod.reset_iteration()
for i in range(mod.opt.loop_per_epoch):
    iterator = Dataset.datasets.dataloader_train.__iter__()
    mod.reset_iteration()
    mod.log_start_train()
    for item in iterator:
        mod.increment_iteration()
        mod.data = EasyDict(item)
        mod.data.points = mod.data.points.to(mod.opt.device)
        if Dataset.datasets.data_augmenter is not None and not Dataset.opt.SVR:
            Dataset.datasets.data_augmenter(Dataset.data.points)
        mod.train_iteration()
        # print(mod.data)
        demo_path = mod.data.image_path[0]
        demo_path = demo_path+'/01.png'
        demo_data = Dataset.datasets.dataset_train.load(demo_path)
        with torch.no_grad():
            mod.demo(EasyDict(demo_data), demo_path)
        break

New MLP decoder : hidden size 512, num_layers 2, activation relu
No network weights to reload!
log/test_code/summary
[0: 1/364] chamfer train loss:  0.33630669116973877 Ellapsed Time: 0.00020082546605004203h ETL: 10.989235330179536h

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.